In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd

In [3]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV

In [4]:
train_df = pd.read_csv('../data/processed/train.csv')
train_df = train_df.drop(['PassengerId'], 1)
train_df.head(2)

,Survived,Pclass,Age,SibSp,Parch,Fare,female,male
0,0,3,22.0,1,0,7.2500,0.0,1.0
1,1,1,38.0,1,0,71.2833,1.0,0.0


In [5]:
test_df = pd.read_csv('../data/processed/test.csv')
test_df.head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,female,male
0,892,3,34.5,0,0,7.8292,0.0,1.0
1,893,3,47.0,1,0,7.0000,1.0,0.0
2,894,2,62.0,0,0,9.6875,0.0,1.0
3,895,3,27.0,0,0,8.6625,0.0,1.0
4,896,3,22.0,1,1,12.2875,1.0,0.0


In [6]:
model = RandomForestClassifier(n_estimators=100)

In [7]:
train_cols = [col for col in train_df.columns if col not in ['Survived']]

In [8]:
Y = train_df['Survived']
X = train_df[train_cols]

In [9]:
best_i = 1
best_model = 0.0

In [18]:
for i in xrange(1, len(train_cols)+1):
    print i
    select = SelectKBest(k = i)
    clf = RandomForestClassifier(n_estimators=200, warm_start = True,
                                 max_depth = 6, 
                                 max_features = 'sqrt')
    pipeline = make_pipeline(select, clf)

    pipeline.fit(X, Y)
    predictions = pipeline.predict(X)
    predict_proba = pipeline.predict_proba(X)[:,1]

    cv_score = cross_validation.cross_val_score(pipeline, X, Y, cv= 10)
    print("Accuracy : %.4g" % metrics.accuracy_score(Y.values, predictions))
    print("AUC Score (Train): %f" % metrics.roc_auc_score(Y, predict_proba))
    print("CV Score : Mean - %.7g | Std - %.7g | Min - %.7g | Max - %.7g" % (np.mean(cv_score), np.std(cv_score), 
            np.min(cv_score),
            np.max(cv_score)))
    print '\n'
    model_score = metrics.accuracy_score(Y.values, predictions) 
    model_score += metrics.roc_auc_score(Y, predict_proba) 
    model_score += metrics.roc_auc_score(Y, predict_proba)
    if model_score > best_model:
        best_model = model_score
        best_i = i

1
Accuracy : 0.7868
AUC Score (Train): 0.766873
CV Score : Mean - 0.7866982 | Std - 0.02794224 | Min - 0.741573 | Max - 0.8426966


2
Accuracy : 0.7868
AUC Score (Train): 0.766873
CV Score : Mean - 0.7866982 | Std - 0.02794224 | Min - 0.741573 | Max - 0.8426966


3
Accuracy : 0.7868
AUC Score (Train): 0.832835
CV Score : Mean - 0.7733399 | Std - 0.03164068 | Min - 0.741573 | Max - 0.8426966


4
Accuracy : 0.853
AUC Score (Train): 0.917537
CV Score : Mean - 0.8181719 | Std - 0.03170869 | Min - 0.7777778 | Max - 0.8764045


5
Accuracy : 0.862
AUC Score (Train): 0.920802
CV Score : Mean - 0.8126671 | Std - 0.04416481 | Min - 0.741573 | Max - 0.8876404


6
Accuracy : 0.8687
AUC Score (Train): 0.935894
CV Score : Mean - 0.8227798 | Std - 0.04841511 | Min - 0.741573 | Max - 0.9101124


7
Accuracy : 0.8653
AUC Score (Train): 0.930757
CV Score : Mean - 0.8249393 | Std - 0.04167071 | Min - 0.752809 | Max - 0.8988764




In [19]:
select = SelectKBest(k = best_i)
clf = RandomForestClassifier(n_estimators=200, warm_start = True,
                             max_depth = 6, 
                             max_features = 'sqrt')
pipeline = make_pipeline(select, clf)

pipeline.fit(X, Y)
predictions = pipeline.predict(X)
predict_proba = pipeline.predict_proba(X)[:,1]

In [20]:
X.head()

,Pclass,Age,SibSp,Parch,Fare,female,male
0,3,22.0,1,0,7.2500,0.0,1.0
1,1,38.0,1,0,71.2833,1.0,0.0
2,3,26.0,0,0,7.9250,1.0,0.0
3,1,35.0,1,0,53.1000,1.0,0.0
4,3,35.0,0,0,8.0500,0.0,1.0


In [21]:
test_noid = test_df[test_df.columns[1:]]

In [22]:
test_df['Survived'] = pipeline.predict(test_noid)

ValueError: X has a different shape than during fitting.

In [15]:
output_cols = ['PassengerId', 'Survived']

In [16]:
output = test_df[output_cols]
output.to_csv('../data/results_rf.csv', index=False)